list_of_symbols = ['NIFTY','BANKNIFTY']

year,month,day,length = 2023,7,13,12

for sym in list_of_symbols:
    td_obj.add_option_symbols(sym, year, month, day,length=length)
time.sleep(5)
data = td_obj.get_oc_data()
syms = list(data.keys())
print(f"Downloaded data for {[print(x) for x in syms]}")

df_dict = {k:pd.DataFrame.from_records(v) for k,v in data.items()}
df_dict['BANKNIFTY2023713']

In [9]:
from  TDClient import FData
import time
import pandas as pd
import datetime
import pytz
indian_tz = pytz.timezone('Asia/Kolkata')
import warnings
warnings.filterwarnings("ignore")
from requests import get, post

username = 'tdwsp353'
password = 'anjan@353'

nurl = 'http://3.6.166.159/td'                 
url = "http://127.0.0.1:5000"
#nurl='http://192.168.2.173:6969'

td_obj = FData(key='trial', base_url=nurl)
td_obj.login()
td_obj.connect_to_server(username=username, password=password, realtime_port=8082, url='push.truedata.in')

<Response [200]>

In [ ]:
import requests

class OrderWebhook:
    def __init__(self):
        self.uri = "http://3.6.166.159/fpt/"

    def placeorder(self, buyorsell, day, month, year, side, strike_price, symbol):
        a = requests.post(self.uri + "makeorder",
                          json={"buyorsell": buyorsell, "day": day, "month": month, "year": year, "side": side,
                                "strike_price": strike_price, "symbol": symbol})
        return a.json()


obj = OrderWebhook()

In [10]:
from sqlalchemy.sql.expression import true

sym = 'BANKNIFTY'
year,month,day,length = 2023,7,27,30

start_time = datetime.time(9, 15)
end_time = datetime.time(14, 50)
current_time = datetime.datetime.now(indian_tz).time()

trade_entry = []
trade_exits = []
live_pnl = []
profit = []
hedge_profit = []

trade = False

In [11]:
td_obj.add_option_symbols(sym, year, month, day,length)
time.sleep(5)

In [ ]:
while (current_time>start_time)&(current_time<end_time):
    
    try:
    
        df = pd.DataFrame(td_obj.get_oc_data()[sym + str(year) + str(month)+ str(day)])
        df = df[['ltp', 'ask','bid', 'ltt', 'strike', 'type']]
        df.index = df['strike']
        df= df.drop(columns =['strike'])
        df.ltt = pd.to_datetime(df.ltt)
        df = df[(df['ask']!=0)&(df['bid']!=0)]

        df_ce = df[df['type'] == 'CE']
        df_pe = df[df['type'] == 'PE']

        diff_df = abs(df_ce['ask'] - df_pe['ask']).sort_values()

        if trade == False:
            print(current_time,diff_df[0])

        if (diff_df[0]<10) & (trade == False):

            ce_premium = df_ce[df_ce.index==diff_df.index[0]]['bid'][0]
            pe_premium = df_pe[df_pe.index==diff_df.index[0]]['bid'][0]

            straddle_strike = diff_df.index[0]
            straddle_premium = ce_premium + pe_premium

            hedge_budget = straddle_premium*0.30

            df_ce['dist_from_strike'] = int(straddle_strike) - df_ce.index.astype(int)
            df_pe['dist_from_strike'] = df_pe.index.astype(int) - int(straddle_strike)

            df_ce['strike_price'] = df_ce.index
            df_pe['strike_price'] = df_pe.index

            df_ce.index = df_ce['dist_from_strike']
            df_pe.index = df_pe['dist_from_strike']

            df_ce = df_ce.rename(columns = {'strike_price':'ce_strike_price', 'ask':'ce_ask', 'bid':'ce_bid'})
            df_pe = df_pe.rename(columns = {'strike_price':'pe_strike_price', 'ask':'pe_ask', 'bid':'pe_bid'})

            df_hedge = pd.concat([df_ce,df_pe], axis = 1)
            df_hedge['hedge_premium'] = df_hedge['ce_ask'] + df_hedge['pe_ask']
            df_hedge = df_hedge.dropna()
            df_hedge['closest_to_budget'] = df_hedge['hedge_premium'] - hedge_budget
            df_hedge = df_hedge[df_hedge['closest_to_budget']<0]
            df_hedge = df_hedge.sort_values(by = 'closest_to_budget', ascending = False)

            ce_hedge_strike = df_hedge['ce_strike_price'].iloc[0]
            ce_hedge_premium = df_hedge['ce_ask'].iloc[0]
            pe_hedge_strike = df_hedge['pe_strike_price'].iloc[0]
            pe_hedge_premium = df_hedge['pe_ask'].iloc[0]
            hedge_premium = ce_hedge_premium + pe_hedge_premium

            if hedge_premium < hedge_budget:

                trade = True

                trade_entry.append([current_time,
                                'Buy',
                                'Straddle Strike Price :',
                                straddle_strike,
                                'Straddle Premium:',
                                straddle_premium,
                                'Hedge CE Strike:',
                                ce_hedge_strike,
                                'Hedge PE Strike:',
                                pe_hedge_strike,
                                'Hedge Premium:',
                                hedge_premium
                                ])
                
                #Hedge Entry
                
                if len(trade_entry) == 1:
                    
                    obj.placeorder(buyorsell="B", day=day, month=month, year=year, 
                                   side="CE", strike_price=trade_entry[0][7], symbol=sym)
                    
                    obj.placeorder(buyorsell="B", day=day, month=month, year=year, 
                                   side="PE", strike_price=trade_entry[0][9], symbol=sym)
                
                time.sleep(5)
                
                #Straddle Entry
                
                obj.placeorder(buyorsell="S", day=day, month=month, year=year, 
                               side="CE", strike_price=trade_entry[-1][3], symbol=sym)
                
                obj.placeorder(buyorsell="S", day=day, month=month, year=year, 
                               side="PE", strike_price=trade_entry[-1][3], symbol=sym)                
                    

                print(trade_entry[-1])

        if trade == True:

            difference_ce_pe = abs(df.loc[straddle_strike]['bid'][0] - df.loc[straddle_strike]['bid'][1])

            sell_straddle_premium = df.loc[straddle_strike]['bid'].sum()

            ceh1 = df.loc[ce_hedge_strike]
            ceh1 = ceh1[ceh1['type'] == 'CE']['bid'][0]

            peh1 = df.loc[pe_hedge_strike]
            peh1 = peh1[peh1['type'] == 'PE']['bid'][0]

            sell_hedge_premium = ceh1 + peh1

            live_profits = (straddle_premium - sell_straddle_premium)*25

            live_pnl.append([current_time,
                             'Trade Number:',
                             len(trade_entry),
                             'Difference CE & PE:'+ str(round(difference_ce_pe,2)),
                             'Live PnL of Straddle:' + str(round(live_profits,2))
                             ])

            print(live_pnl[-1])

        if trade == True:

            if (abs(df.loc[straddle_strike]['bid'][0] - df.loc[straddle_strike]['bid'][1])>100):

                sell_straddle_premium = df.loc[straddle_strike]['ask'].sum()

                ceh1 = df.loc[ce_hedge_strike]
                ceh1 = ceh1[ceh1['type'] == 'CE']['bid'][0]

                peh1 = df.loc[pe_hedge_strike]
                peh1 = peh1[peh1['type'] == 'PE']['bid'][0]

                sell_hedge_premium = ceh1 + peh1

                trade_exits.append([current_time,
                                    'Sell',
                                    'Straddle Sell Premium:' + str(sell_straddle_premium),
                                    'Hedge Sell Premium:' + str(sell_hedge_premium)
                                    ])
                
                #Straddle Exit
                
                obj.placeorder(buyorsell="B", day=day, month=month, year=year, 
                               side="CE", strike_price=trade_entry[-1][3], symbol=sym)
                
                obj.placeorder(buyorsell="B", day=day, month=month, year=year, 
                               side="PE", strike_price=trade_entry[-1][3], symbol=sym) 

                pf = (straddle_premium - sell_straddle_premium)*25

                profit.append([current_time,
                               'Profit :',
                               pf])

                trade = False

                print(trade_exits[-1])
                print(profit[-1])
                print('*' * 125)
                print("\n")

        if trade == True:

            if current_time>=datetime.time(14,49):

                sell_straddle_premium = df.loc[straddle_strike]['ask'].sum()

                ce_hedge_strike = trade_entry[0][7]
                pe_hedge_strike = trade_entry[0][9]

                ceh1 = df.loc[ce_hedge_strike]
                ceh1 = ceh1[ceh1['type'] == 'CE']['bid'][0]

                peh1 = df.loc[pe_hedge_strike]
                peh1 = peh1[peh1['type'] == 'PE']['bid'][0]

                sell_hedge_premium = ceh1 + peh1
                
                hedge_premium = trade_entry[0][-1]

                trade_exits.append([current_time,
                                    'Sell',
                                    'Straddle Sell Premium:' + str(sell_straddle_premium),
                                    'Hedge Sell Premium:' + str(sell_hedge_premium)
                                    ])

                pf = (straddle_premium - sell_straddle_premium)*25

                profit.append([current_time,
                               'Profit :',
                               pf])

                hedge_profit.append([current_time,
                                   'Profit :',
                                    (sell_hedge_premium-hedge_premium)*25,
                                    ])
                
                #Straddle Exit
                
                
                obj.placeorder(buyorsell="B", day=day, month=month, year=year, 
                               side="CE", strike_price=trade_entry[-1][3], symbol=sym)
                
                obj.placeorder(buyorsell="B", day=day, month=month, year=year, 
                               side="PE", strike_price=trade_entry[-1][3], symbol=sym)
                
                time.sleep(5)
                
                
                #Hedges Exit
                
                obj.placeorder(buyorsell="S", day=day, month=month, year=year, 
                                side="CE", strike_price=trade_entry[0][7], symbol=sym)
                    
                obj.placeorder(buyorsell="S", day=day, month=month, year=year, 
                                side="PE", strike_price=trade_entry[0][9], symbol=sym)

                trade = False

                print(trade_exits[-1])
                print(profit[-1])
                print('*' * 125)
                print("\n")

                break

        time.sleep(5)
        current_time = datetime.datetime.now(indian_tz).time()
    
    except Exception as e:
        
        td_obj = FData(key='trial', base_url=nurl)
        td_obj.login()
        td_obj.connect_to_server(username=username, password=password, realtime_port=8082, url='push.truedata.in')
        
        td_obj.add_option_symbols(sym, year, month, day,length)
        time.sleep(5)
        
        print("Restart" + ' '+str(e))
        pass

In [13]:
while (current_time>start_time)&(current_time<end_time):
    
    try:
    
        df = pd.DataFrame(td_obj.get_oc_data()[sym + str(year) + str(month)+ str(day)])
        df = df[['ltp', 'ask','bid', 'ltt', 'strike', 'type']]
        df.index = df['strike']
        df= df.drop(columns =['strike'])
        df.ltt = pd.to_datetime(df.ltt)
        df = df[(df['ask']!=0)&(df['bid']!=0)]

        df_ce = df[df['type'] == 'CE']
        df_pe = df[df['type'] == 'PE']

        diff_df = abs(df_ce['ask'] - df_pe['ask']).sort_values()

        if trade == False:
            print(current_time,diff_df[0])

        if (diff_df[0]<10) & (trade == False):

            ce_premium = df_ce[df_ce.index==diff_df.index[0]]['bid'][0]
            pe_premium = df_pe[df_pe.index==diff_df.index[0]]['bid'][0]

            straddle_strike = diff_df.index[0]
            straddle_premium = ce_premium + pe_premium

            hedge_budget = straddle_premium*0.30

            df_ce['dist_from_strike'] = int(straddle_strike) - df_ce.index.astype(int)
            df_pe['dist_from_strike'] = df_pe.index.astype(int) - int(straddle_strike)

            df_ce['strike_price'] = df_ce.index
            df_pe['strike_price'] = df_pe.index

            df_ce.index = df_ce['dist_from_strike']
            df_pe.index = df_pe['dist_from_strike']

            df_ce = df_ce.rename(columns = {'strike_price':'ce_strike_price', 'ask':'ce_ask', 'bid':'ce_bid'})
            df_pe = df_pe.rename(columns = {'strike_price':'pe_strike_price', 'ask':'pe_ask', 'bid':'pe_bid'})

            df_hedge = pd.concat([df_ce,df_pe], axis = 1)
            df_hedge['hedge_premium'] = df_hedge['ce_ask'] + df_hedge['pe_ask']
            df_hedge = df_hedge.dropna()
            df_hedge['closest_to_budget'] = df_hedge['hedge_premium'] - hedge_budget
            df_hedge = df_hedge[df_hedge['closest_to_budget']<0]
            df_hedge = df_hedge.sort_values(by = 'closest_to_budget', ascending = False)

            ce_hedge_strike = df_hedge['ce_strike_price'].iloc[0]
            ce_hedge_premium = df_hedge['ce_ask'].iloc[0]
            pe_hedge_strike = df_hedge['pe_strike_price'].iloc[0]
            pe_hedge_premium = df_hedge['pe_ask'].iloc[0]
            hedge_premium = ce_hedge_premium + pe_hedge_premium

            if hedge_premium < hedge_budget:

                trade = True

                trade_entry.append([current_time,
                                'Buy',
                                'Straddle Strike Price :',
                                straddle_strike,
                                'Straddle Premium:',
                                straddle_premium,
                                'Hedge CE Strike:',
                                ce_hedge_strike,
                                'Hedge PE Strike:',
                                pe_hedge_strike,
                                'Hedge Premium:',
                                hedge_premium
                                ])
                

                
               
                    

                print(trade_entry[-1])

        if trade == True:

            difference_ce_pe = abs(df.loc[straddle_strike]['bid'][0] - df.loc[straddle_strike]['bid'][1])

            sell_straddle_premium = df.loc[straddle_strike]['bid'].sum()

            ceh1 = df.loc[ce_hedge_strike]
            ceh1 = ceh1[ceh1['type'] == 'CE']['bid'][0]

            peh1 = df.loc[pe_hedge_strike]
            peh1 = peh1[peh1['type'] == 'PE']['bid'][0]

            sell_hedge_premium = ceh1 + peh1

            live_profits = (straddle_premium - sell_straddle_premium)*25

            live_pnl.append([current_time,
                             'Trade Number:',
                             len(trade_entry),
                             'Difference CE & PE:'+ str(round(difference_ce_pe,2)),
                             'Live PnL of Straddle:' + str(round(live_profits,2))
                             ])

            print(live_pnl[-1])

        if trade == True:

            if (abs(df.loc[straddle_strike]['bid'][0] - df.loc[straddle_strike]['bid'][1])>150):

                sell_straddle_premium = df.loc[straddle_strike]['ask'].sum()

                ceh1 = df.loc[ce_hedge_strike]
                ceh1 = ceh1[ceh1['type'] == 'CE']['bid'][0]

                peh1 = df.loc[pe_hedge_strike]
                peh1 = peh1[peh1['type'] == 'PE']['bid'][0]

                sell_hedge_premium = ceh1 + peh1

                trade_exits.append([current_time,
                                    'Sell',
                                    'Straddle Sell Premium:' + str(sell_straddle_premium),
                                    'Hedge Sell Premium:' + str(sell_hedge_premium)
                                    ])
                
 

                pf = (straddle_premium - sell_straddle_premium)*25

                profit.append([current_time,
                               'Profit :',
                               pf])

                trade = False

                print(trade_exits[-1])
                print(profit[-1])
                print('*' * 125)
                print("\n")
                
        if len(trade_entry)>=1:
            
            ce_hedge_strike = trade_entry[0][7]
            pe_hedge_strike = trade_entry[0][9]

            ceh1 = df.loc[ce_hedge_strike]
            ceh1 = ceh1[ceh1['type'] == 'CE']['bid'][0]

            peh1 = df.loc[pe_hedge_strike]
            peh1 = peh1[peh1['type'] == 'PE']['bid'][0]

            sell_hedge_premium = ceh1 + peh1

            hedge_premium = trade_entry[0][-1]
            
            hpf = (sell_hedge_premium-hedge_premium)*25
            
            print(current_time,'Hedge Live Profits: ', hpf)
            print("\n")

        if trade == True:

            if current_time>=datetime.time(14,49):

                sell_straddle_premium = df.loc[straddle_strike]['ask'].sum()

                ce_hedge_strike = trade_entry[0][7]
                pe_hedge_strike = trade_entry[0][9]

                ceh1 = df.loc[ce_hedge_strike]
                ceh1 = ceh1[ceh1['type'] == 'CE']['bid'][0]

                peh1 = df.loc[pe_hedge_strike]
                peh1 = peh1[peh1['type'] == 'PE']['bid'][0]

                sell_hedge_premium = ceh1 + peh1
                
                hedge_premium = trade_entry[0][-1]

                trade_exits.append([current_time,
                                    'Sell',
                                    'Straddle Sell Premium:' + str(sell_straddle_premium),
                                    'Hedge Sell Premium:' + str(sell_hedge_premium)
                                    ])

                pf = (straddle_premium - sell_straddle_premium)*25

                profit.append([current_time,
                               'Profit :',
                               pf])

                hedge_profit.append([current_time,
                                   'Profit :',
                                    (sell_hedge_premium-hedge_premium)*25,
                                    ])
                


                trade = False

                print(trade_exits[-1])
                print(profit[-1])
                print('*' * 125)
                print("\n")

                break

        time.sleep(10)
        current_time = datetime.datetime.now(indian_tz).time()
    
    except Exception as e:
        
        td_obj = FData(key='trial', base_url=nurl)
        td_obj.login()
        td_obj.connect_to_server(username=username, password=password, realtime_port=8082, url='push.truedata.in')
        
        td_obj.add_option_symbols(sym, year, month, day,length)
        time.sleep(5)
        
        print("Restart" + ' '+str(e))
        pass

[datetime.time(13, 35, 27, 260980), 'Trade Number:', 1, 'Difference CE & PE:11.55', 'Live PnL of Straddle:1510.0']
13:35:27.260980 Hedge Live Profits:  -970.0000000000001


[datetime.time(13, 36, 12, 379012), 'Trade Number:', 1, 'Difference CE & PE:9.4', 'Live PnL of Straddle:1501.25']
13:36:12.379012 Hedge Live Profits:  -960.0


[datetime.time(13, 36, 22, 535352), 'Trade Number:', 1, 'Difference CE & PE:4.65', 'Live PnL of Straddle:1520.0']
13:36:22.535352 Hedge Live Profits:  -958.7500000000002


[datetime.time(13, 36, 32, 710137), 'Trade Number:', 1, 'Difference CE & PE:10.8', 'Live PnL of Straddle:1508.75']
13:36:32.710137 Hedge Live Profits:  -963.7499999999999


[datetime.time(13, 36, 42, 910409), 'Trade Number:', 1, 'Difference CE & PE:6.2', 'Live PnL of Straddle:1506.25']
13:36:42.910409 Hedge Live Profits:  -962.5


[datetime.time(13, 36, 53, 120728), 'Trade Number:', 1, 'Difference CE & PE:12.3', 'Live PnL of Straddle:1496.25']
13:36:53.120728 Hedge Live Profits:  -963.75000

In [14]:
profit

[[datetime.time(14, 49, 2, 383272), 'Profit :', 2054.9999999999995]]

In [15]:
hedge_profit

[[datetime.time(14, 49, 2, 383272), 'Profit :', -1212.5]]

In [21]:
print('bnf', 'Profit :', pd.concat([pd.DataFrame(profit), pd.DataFrame(hedge_profit)], axis = 0)[2].sum(),'\n','Shiftings : ', len(profit))

bnf Profit : 842.4999999999995 
 Shiftings :  1
